In [6]:
##############################################################################
#
#            Requirements:
#              - Python 3.5+
#              - Exiv2 installation
# 
##############################################################################

def read_config_values():
    """Reads values from the configuration files and returns dictionary"""
    
    if config_file_exists():
        import image_caption_config
    else:
        raise ValueError ('Missing file image_caption_config.py, please refer to documentation') 
    
    # Values in a hidden config file override the config file, useful to prevent user config settings being pushed to github
    if hidden_config_file_exists():
        import _image_caption_config
        if image_caption_config.config.keys() != _image_caption_config.config.keys():
            raise ValueError ('keys in config files dont match hidden config file') 
        else:
            config_values = _image_caption_config.config
    else:
        config_values = image_caption_config.config
    
    return (config_values)


def config_file_exists():
    """ Check if a configuration file exists"""
    return(os.path.isfile('image_caption_config.py'))


def hidden_config_file_exists():
    """ Check if hidden configuration file exists"""
    return(os.path.isfile('_image_caption_config.py'))


def in_dir_exists(in_dir):
    """Check if in_dir directory exists"""
    return(os.path.isdir(in_dir))
            
            
def dir_exists(dir):
    """Check if out_dir directory exists"""
    return(os.path.isdir(dir))


def create_dir_and_parents(dir):
    """Create out_dir"""
    pathlib.Path(dir).mkdir(parents=True)

    
def dir_is_empty(dir):
    return (len(os.listdir(dir)) == 0)


class CaptionedPhoto:
    
    def __init__(self, file_path, img_ratios):
        """Opens image, sets the frame and border ratios and transpose if required"""
        self.original_image = Image.open(file_path)
        self.frame_ratio, self.borders_ratio = img_ratios
        self._filepath = file_path
        
        # If orientation is stored via exif metadata then transpose image and remove the orientation metadata
        self.original_image = ImageOps.exif_transpose(self.original_image)

        
    def _title(self):
        """Returns the XMP description (shows as title in Windows Explorer of self.original_image """
        result = subprocess.run(['exiv2', '-g', 'Xmp.dc.description', '-Pv', self._filepath], stdout=subprocess.PIPE)
        title = result.stdout.decode(encoding="utf-8").strip()
        
        # remove text similiar to this 'lang="<something>"' that occurs at start of the string
        title = re.sub('lang=".*" ', '', title)
        
        if inputs['caption_prefix'] == '':
            return (title)
        elif title[:len(inputs['caption_prefix'])] == inputs['caption_prefix']:
            return(title[-len(title) + len(inputs['caption_prefix']):].strip())
        else:
            return ''

        
    def _max_side_length(self):
        """returns the max of height and width of the image"""
        img_width, img_height = self.original_image.size
        return(max(img_width, img_height))

    
    def _original_image_width(self):
        """Returns the original image width"""
        return self.original_image.size[0]

    
    def _original_image_height(self):
        """Returns the original image height"""
        return self.original_image.size[1]

        
    def frame_width(self):
        """Returns frame width"""
        return(int(self._max_side_length() * self.frame_ratio))
    
    
    def borders_width(self):
        """Returns widths of borders"""
        return(int(self._max_side_length() * self.borders_ratio))

    
    def _border_image(self):
        """Returns rectangle reprepsenting border image excluding photo and frame"""
        border_image_width = self._original_image_width() + self.borders_width() * 2
        border_image_height = self._original_image_height() + self.borders_width() * 2
        img_border = Image.new('RGB', (border_image_width, border_image_height), inputs['border_colour'])
        return(img_border)
    
    
    def _border_image_width(self):
        """Returns the border image width"""
        return self._border_image().size[0]

    
    def _border_image_height(self):
        """Returns the border image height"""
        return self._border_image().size[1]
    
    
    def _frame_image(self):
        """Returns rectangle representing the frame image excluding border and photo"""
        frame_img_width = self._border_image_width() + self.frame_width()*2
        frame_img_height = self._border_image_height() + self.frame_width()*2
        img_frame = Image.new('RGB', (frame_img_width, frame_img_height), inputs['frame_colour'])
        return(img_frame)
        
        
    def _captioned_image(self):
        """ Returns orginal image with added frame and boreder"""
        bordered_image = self._border_image()
        bordered_image.paste(self.original_image, (self.borders_width(), self.borders_width()))
        framed_image = self._frame_image()
        framed_image.paste(bordered_image, (self.frame_width(), self.frame_width()))

        # !!!!!!!!!! NEED to adjust font size to config!!!!!!!!!!!!!
        caption_font = ImageFont.truetype(inputs['caption_font'], 60)
        x = framed_image.size[0]/2
        y =framed_image.size[1] - self.frame_width() -self.borders_width() /2
        ImageDraw.Draw(framed_image).text(xy=(x,y), text=self._title(), font=caption_font, fill =inputs['font_colour'], anchor='mm')
        
        return (framed_image)
        
    def _output_image(self):
        """Returns original image inside border, indside frame if image has a valid caption"""
        
        if self._title() !='':
            return (self._captioned_image())
        else:
            return (self.original_image)
        
        
    def save(self, path):
        """Saves the captioned image to path directory"""
        self._output_image().save(path)

        
if __name__ == "__main__":
    
    from PIL import Image, ImageDraw, ImageFont, ImageOps
    import os, pathlib, subprocess, re
  
    # Read configuration inputs into a dictionary
    inputs = read_config_values()
    
    if not dir_exists(inputs['out_dir']):
        create_dir_and_parents(inputs['out_dir'])
    elif not dir_is_empty(inputs['out_dir']):
        raise ValueError ('Output directory (' +  inputs['out_dir'] +') is not empty') 
    
    for filename_and_extension in os.listdir(inputs['in_dir']):
        image_extensions_lower_case = ['.jpeg', '.jpg', '.png']
        base_filename, file_extension = os.path.splitext(filename_and_extension)        
        if file_extension.lower() in image_extensions_lower_case:
            input_filepath = os.path.join(inputs['in_dir'], base_filename + file_extension)
            print ('processing ' + input_filepath)
            output_filepath = os.path.join(inputs['out_dir'], base_filename + file_extension)
            my_photo = CaptionedPhoto(input_filepath, inputs['image_ratios'])
            my_photo.save(output_filepath)
    print('\nProcessing complete...')

processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/20230408_182048.jpg
processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/Landscape_Example.JPG
processing /home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/Portrait_Example.jpg

Processing complete...
